# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 21 2022 11:29AM,249148,19,8675493,VITABIOTICS USA INC.,Released
1,Oct 21 2022 11:21AM,249147,19,8675489,VITABIOTICS USA INC.,Released
2,Oct 21 2022 11:17AM,249145,15,PNC398025A,"Person & Covey, Inc.",Executing
3,Oct 21 2022 11:17AM,249145,15,PNC398099A,"Person & Covey, Inc.",Released
4,Oct 21 2022 11:17AM,249145,15,PNC398116A,"Person & Covey, Inc.",Executing
5,Oct 21 2022 11:17AM,249145,15,PNC398117A,"Person & Covey, Inc.",Released
6,Oct 21 2022 11:17AM,249145,15,PNC394440A,"Person & Covey, Inc.",Released
7,Oct 21 2022 11:17AM,249145,15,PNC394593A,"Person & Covey, Inc.",Released
8,Oct 21 2022 11:17AM,249145,15,VT80179565,"Virtus Pharmaceuticals, LLC",Released
9,Oct 21 2022 11:17AM,249145,15,VT80179640,"Virtus Pharmaceuticals, LLC",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,249145,Executing,8
22,249145,Released,11
23,249146,Released,1
24,249147,Released,1
25,249148,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249144,NaN,NaN,1.0
249145,NaN,8.0,11.0
249146,NaN,NaN,1.0
249147,NaN,NaN,1.0
249148,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249041,0.0,1.0,0.0
249082,11.0,0.0,3.0
249086,0.0,0.0,1.0
249088,0.0,0.0,1.0
249115,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249041,0,1,0
249082,11,0,3
249086,0,0,1
249088,0,0,1
249115,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249041,0,1,0
1,249082,11,0,3
2,249086,0,0,1
3,249088,0,0,1
4,249115,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249041,,1,
1,249082,11,,3
2,249086,,,1
3,249088,,,1
4,249115,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc."
8,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC"
18,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc."
21,Oct 21 2022 11:15AM,249146,10,Methapharm-G
22,Oct 21 2022 11:03AM,249144,10,"Senseonics, Incorporated"
23,Oct 21 2022 10:59AM,249137,10,Bio-PRF
27,Oct 21 2022 10:59AM,249143,19,ACG North America LLC
28,Oct 21 2022 10:55AM,249142,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,,,1
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.,,,1
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc.",,8,11
3,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC",,8,11
4,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc.",,8,11
5,Oct 21 2022 11:15AM,249146,10,Methapharm-G,,,1
6,Oct 21 2022 11:03AM,249144,10,"Senseonics, Incorporated",,,1
7,Oct 21 2022 10:59AM,249137,10,Bio-PRF,,,4
8,Oct 21 2022 10:59AM,249143,19,ACG North America LLC,,,1
9,Oct 21 2022 10:55AM,249142,10,Methapharm-G,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,1,,
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.,1,,
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc.",11,8,
3,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC",11,8,
4,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc.",11,8,
5,Oct 21 2022 11:15AM,249146,10,Methapharm-G,1,,
6,Oct 21 2022 11:03AM,249144,10,"Senseonics, Incorporated",1,,
7,Oct 21 2022 10:59AM,249137,10,Bio-PRF,4,,
8,Oct 21 2022 10:59AM,249143,19,ACG North America LLC,1,,
9,Oct 21 2022 10:55AM,249142,10,Methapharm-G,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,1,,
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.,1,,
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc.",11,8,
3,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC",11,8,
4,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc.",11,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,1.0,NaN,NaN
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.,1.0,NaN,NaN
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc.",11.0,8.0,NaN
3,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC",11.0,8.0,NaN
4,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc.",11.0,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,1.0,0.0,0.0
1,Oct 21 2022 11:21AM,249147,19,VITABIOTICS USA INC.,1.0,0.0,0.0
2,Oct 21 2022 11:17AM,249145,15,"Person & Covey, Inc.",11.0,8.0,0.0
3,Oct 21 2022 11:17AM,249145,15,"Virtus Pharmaceuticals, LLC",11.0,8.0,0.0
4,Oct 21 2022 11:17AM,249145,15,"Alliance Pharma, Inc.",11.0,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2242176,52.0,15.0,11.0
12,498274,1.0,1.0,0.0
15,747435,33.0,24.0,0.0
16,249086,1.0,0.0,0.0
19,996569,3.0,1.0,0.0
21,996383,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2242176,52.0,15.0,11.0
1,12,498274,1.0,1.0,0.0
2,15,747435,33.0,24.0,0.0
3,16,249086,1.0,0.0,0.0
4,19,996569,3.0,1.0,0.0
5,21,996383,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,15.0,11.0
1,12,1.0,1.0,0.0
2,15,33.0,24.0,0.0
3,16,1.0,0.0,0.0
4,19,3.0,1.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,1.0
2,15,Released,33.0
3,16,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0
Executing,15.0,1.0,24.0,0.0,1.0,1.0
Released,52.0,1.0,33.0,1.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0
1,Executing,15.0,1.0,24.0,0.0,1.0,1.0
2,Released,52.0,1.0,33.0,1.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0
1,Executing,15.0,1.0,24.0,0.0,1.0,1.0
2,Released,52.0,1.0,33.0,1.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()